In [1]:
# Import necessary libraries
import pandas as pd
import os
import numpy as np
from geopy.geocoders import Nominatim
import pycountry
import time

# Set pandas display option
pd.set_option('display.max_columns', None)

# Define file paths
cwd = os.getcwd()
path_asyl = os.path.join(cwd, 'data', 'query_data-2', 'asylum-applications.csv')
path_wdi = os.path.join(cwd, 'data', 'query_data-5', 'wdi.csv')

# Read asylum applications data
df_asyl = pd.read_csv(path_asyl)


# Read WDI data
df_wdi = pd.read_csv(path_wdi)


# Group asylum applications data
df_asyl_grp = df_asyl.groupby(by=['Year','Country of origin (ISO)', 'Country of asylum (ISO)']).sum(numeric_only=True).reset_index().sort_values(by='Year', ascending=False)

# Merge grouped asylum data with WDI data (Country of origin)
df_merge = df_asyl_grp.merge(df_wdi, how='left', left_on=['Year', 'Country of origin (ISO)'], right_on=['year' ,'iso3'])
df_merge.rename(columns={
    'iso3': 'iso3_origin',
    'iso2': 'iso2_origin',
    'name': 'name_origin',
    'capital': 'capital_origin',
    'continent': 'continent_origin',
    'region': 'region_origin',
    'income_level': 'income_level_origin',
    'population': 'population_origin',
    'gdp': 'gdp_origin',
    'inflation': 'inflation_origin',
    'female_population': 'female_population_origin',
    'life_expectancy': 'life_expectancy_origin',
    'poverty': 'poverty_origin',
    'fertility': 'fertility_origin'
}, inplace=True)

# Drop unnecessary columns
df_merge.drop(columns=[
    'hospital_beds_1000', 'physicians_1000', 'maternal_death_risk',
    'suicides_100k', 'health_expenditure_share', 'infant_mortality_1000',
    'renewables_share', 'co2_capita', 'greenhousegas_capita'
], inplace=True)

# Merge with WDI data again (Country of asylum)
df_merge = df_merge.merge(df_wdi, how='left', left_on=['Year', 'Country of asylum (ISO)'], right_on=['year' ,'iso3'])
df_merge.drop(columns=[
    'hospital_beds_1000', 'physicians_1000', 'maternal_death_risk',
    'suicides_100k', 'health_expenditure_share', 'infant_mortality_1000',
    'renewables_share', 'co2_capita', 'greenhousegas_capita', 'year_x', 'year_y'
], inplace=True)

# Rename columns for Country of asylum
df_merge.rename(columns={
    'iso3': 'iso3_asylum',
    'iso2': 'iso2_asylum',
    'name': 'name_asylum',
    'capital': 'capital_asylum',
    'continent': 'continent_asylum',
    'region': 'region_asylum',
    'income_level': 'income_level_asylum',
    'population': 'population_asylum',
    'gdp': 'gdp_asylum',
    'inflation': 'inflation_asylum',
    'female_population': 'female_population_asylum',
    'life_expectancy': 'life_expectancy_asylum',
    'poverty': 'poverty_asylum',
    'fertility': 'fertility_asylum'
}, inplace=True)


# Filter data for Venezuela
#df_venez = df_merge.loc[df_merge.name_origin == 'Venezuela, RB']

# Function to get latitude and longitude using ISO code
def get_lat_lon(iso_code):
    print(f'Get geodata for: {iso_code}')
    country = pycountry.countries.get(alpha_3=iso_code)
    time.sleep(0.5)
    if country is None:
        return None
    geolocator = Nominatim(user_agent="FHExercise")
    location = geolocator.geocode(country.name)
    if location is None:
        return None
    return location.latitude, location.longitude

# Extract unique ISO codes
# unique_iso_code_asylum = df_venez['Country of asylum (ISO)'].unique()
# unique_iso_code_origin = df_venez['Country of origin (ISO)'].unique()
unique_iso_code_asylum = df_merge['Country of asylum (ISO)'].unique()
unique_iso_code_origin = df_merge['Country of origin (ISO)'].unique()
df_unique_iso_code_asylum = pd.DataFrame(unique_iso_code_asylum, columns=['iso_origin'])
df_unique_iso_code_origin = pd.DataFrame(unique_iso_code_origin, columns=['iso_origin'])
df_iso_code = pd.concat([df_unique_iso_code_asylum, df_unique_iso_code_origin]).reset_index(drop=True).rename(columns={'iso_origin': 'iso_code'})


# Get latitude and longitude for each ISO code
df_iso_code['iso_lat_long'] = df_iso_code['iso_code'].apply(get_lat_lon)

# Save ISO codes with lat/long to CSV
df_iso_code.to_csv('iso_lat_long.csv', index=False)


Get geodata for: ZMB
Get geodata for: CIV
Get geodata for: DNK
Get geodata for: DEU
Get geodata for: CZE
Get geodata for: CYP
Get geodata for: CHE
Get geodata for: CAN
Get geodata for: BEL
Get geodata for: AUT
Get geodata for: USA
Get geodata for: RUS
Get geodata for: NOR
Get geodata for: IRL
Get geodata for: FRA
Get geodata for: ESP
Get geodata for: ITA
Get geodata for: EST
Get geodata for: FIN
Get geodata for: BRA
Get geodata for: ARG
Get geodata for: SWE
Get geodata for: PRT
Get geodata for: POL
Get geodata for: NLD
Get geodata for: MLT
Get geodata for: MDA
Get geodata for: LUX
Get geodata for: ISR
Get geodata for: ISL
Get geodata for: GRC
Get geodata for: GBR
Get geodata for: CMR
Get geodata for: RWA
Get geodata for: ROU
Get geodata for: NER
Get geodata for: MYS
Get geodata for: MWI
Get geodata for: MAR
Get geodata for: LBY
Get geodata for: LBN
Get geodata for: KOR
Get geodata for: KEN
Get geodata for: IND
Get geodata for: SDN
Get geodata for: SOM
Get geodata for: SSD
Get geodata f